In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorboard
import streamlit as st
import pickle

In [2]:
data = pd.read_csv('D:\study material\My Projects\datasets\Churn_Modelling.csv')
data.head(4)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0


In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
label_enc_gen = LabelEncoder()
data['Gender'] = label_enc_gen.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe_geo = OneHotEncoder()
geo_encode = ohe_geo.fit_transform(data[['Geography']])
geo_encode

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [6]:
ohe_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_df = pd.DataFrame(geo_encode.toarray(),columns = ohe_geo.get_feature_names_out(['Geography']))
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
data = pd.concat([data.drop('Geography',axis=1),geo_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
#Save encoders and scalers
with open('label_enc_gen.pkl','wb') as file:
    pickle.dump(label_enc_gen,file)

with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file)

In [10]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
x = data.drop('Exited',axis=1)
y = data['Exited']

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [19]:
with open('sc.pkl','wb') as file:
    pickle.dump(sc,file)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from keras_tuner import GridSearch

In [21]:
def build_model(hp):
    model = Sequential()
    
    # --- LAYER 1 ---
    units_1 = hp.Int('units_1', min_value=32, max_value=256, step=32)
    activation_1 = hp.Choice('activation_1', ['relu', 'sigmoid', 'leaky_relu'])
    if activation_1 == 'leaky_relu':
        model.add(Dense(units=units_1, input_shape=(x_train.shape[1])))
        alpha_1 = hp.Float('alpha_1', 0.01, 0.3, step=0.05)
        model.add(LeakyReLU(alpha=alpha_1))
    else:
        model.add(Dense(units=units_1, activation=activation_1, input_shape=(x_train.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout_1', 0.1, 0.5, step=0.1)))

    # --- LAYER 2 ---
    units_2 = hp.Int('units_2', min_value=32, max_value=256, step=32)
    activation_2 = hp.Choice('activation_2', ['relu', 'sigmoid', 'leaky_relu'])
    if activation_2 == 'leaky_relu':
        model.add(Dense(units=units_2))
        alpha_2 = hp.Float('alpha_2', 0.01, 0.3, step=0.05)
        model.add(LeakyReLU(alpha=alpha_2))
    else:
        model.add(Dense(units=units_2, activation=activation_2))
    model.add(Dropout(rate=hp.Float('dropout_2', 0.1, 0.5, step=0.1)))

    # --- LAYER 3 ---
    units_3 = hp.Int('units_3', min_value=32, max_value=256, step=32)
    activation_3 = hp.Choice('activation_3', ['relu', 'sigmoid', 'leaky_relu'])
    if activation_3 == 'leaky_relu':
        model.add(Dense(units=units_3))
        alpha_3 = hp.Float('alpha_3', 0.01, 0.3, step=0.05)
        model.add(LeakyReLU(alpha=alpha_3))
    else:
        model.add(Dense(units=units_3, activation=activation_3))
    model.add(Dropout(rate=hp.Float('dropout_3', 0.1, 0.5, step=0.1)))

    # --- OUTPUT ---
    model.add(Dense(1, activation='sigmoid'))

    # --- COMPILE ---
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.1, 0.01, 0.001, 0.0001])),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [22]:
tuner = GridSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_tuning_logs',
    project_name='ann_with_relu_leakyrelu',
    overwrite = True
)

In [23]:
tuner.search(
    x_train, 
    y_train, 
    epochs=20, 
    validation_split=0.2
)

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.8687499761581421

Best val_accuracy So Far: 0.8787500262260437
Total elapsed time: 00h 00m 56s


In [24]:
best_hps = tuner.get_best_hyperparameters()[0]

In [25]:
print("Best hyperparameters found:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

Best hyperparameters found:
units_1: 32
activation_1: relu
dropout_1: 0.1
units_2: 32
activation_2: relu
dropout_2: 0.1
units_3: 32
activation_3: relu
dropout_3: 0.1
learning_rate: 0.01


In [26]:
from datetime import datetime

In [27]:
from tensorflow.keras.callbacks import TensorBoard
log_dir = "logs/fit" +datetime.now().strftime("%Y%m%d-%H%M%S")

In [28]:
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [29]:
final_model = build_model(best_hps)

In [30]:
final_model.fit(x_train, y_train, epochs=20, batch_size=32, validation_split=0.2,callbacks=tensorflow_callback)

Epoch 1/20
200/200 [==============================] - 1s 2ms/step - loss: 0.4440 - accuracy: 0.8139 - val_loss: 0.3726 - val_accuracy: 0.8581
Epoch 2/20
200/200 [==============================] - 0s 1ms/step - loss: 0.3817 - accuracy: 0.8405 - val_loss: 0.3509 - val_accuracy: 0.8631
Epoch 3/20
200/200 [==============================] - 0s 1ms/step - loss: 0.3693 - accuracy: 0.8478 - val_loss: 0.3401 - val_accuracy: 0.8625
Epoch 4/20
200/200 [==============================] - 0s 1ms/step - loss: 0.3679 - accuracy: 0.8487 - val_loss: 0.3457 - val_accuracy: 0.8587
Epoch 5/20
200/200 [==============================] - 0s 1ms/step - loss: 0.3635 - accuracy: 0.8514 - val_loss: 0.3384 - val_accuracy: 0.8700
Epoch 6/20
200/200 [==============================] - 0s 1ms/step - loss: 0.3583 - accuracy: 0.8523 - val_loss: 0.3243 - val_accuracy: 0.8706
Epoch 7/20
200/200 [==============================] - 0s 1ms/step - loss: 0.3564 - accuracy: 0.8541 - val_loss: 0.3330 - val_accuracy: 0.8725
Epoch 

In [31]:
final_model.save('final_model_ann.keras', save_format='keras')